# Myths

### Myth 1: You Only Deploy One or Two ML Models at a Time
Many people from academic backgrounds tend to also think of ML production in the context of a single model. In reality, companies have many, many ML models. An application might have many different features, and each feature might require its own model.

### Myth 2: If We Don’t Do Anything, Model Performance Remains the Same
Software doesn’t age like fine wine. It ages poorly. The phenomenon in which a software program degrades over time even if nothing seems to have changed is known as ***software rot*** or ***bit rot.***

ML systems suffer from what are known as data ***distribution shifts***, when the data distribution your model encounters in production is different from the data distribution it was trained on.

### Myth 3: You Won’t Need to Update Your Models as Much
Since a model’s performance decays over time, we want to update it as fast as possible. This is an area of ML where we should learn from existing DevOps best practices.

### Myth 4: Most ML Engineers Don’t Need to Worry About Scale
What “scale” means varies from application to application, but examples include a system that serves hundreds of queries per second or millions of users a month.

# Batch Prediction vs Online Prediction

![batch-prediction](./screenshots/batch-prediction.png)
![online-prediction](./screenshots/online-prediction.png)

![batch-vs-online](./screenshots/batch-vs-online.png)

In online prediction, model predicts the result only when the request arrives. A problem with that kind of prediction is that your model might take too long to generate response. Instead of generating predictions as soon as they arrive, what if you compute predictions in advance and store them in your database, and fetch them when requests arrive? batch prediction can also be seen as a trick to reduce the inference latency of more complex models

However, the problem with batch prediction is that it makes your model less responsive to users’ change preferences. This limitation can be seen even in more technologically progressive companies like Netflix. Say you’ve been watching a lot of horror movies lately, so when you first log in to Netflix, horror movies dominate recommendations. But you’re feeling bright today, so you search “comedy” and start browsing the comedy category.

Another problem with batch prediction is that you need to know what requests to generate predictions for in advance. In the case of recommending movies for users, you know in advance how many users to generate recommendations for. However, for cases when you have unpredictable queries—if you have a system to translate from English to French, it might be impossible to anticipate every possible English text to be translated—you need to use online prediction to generate predictions as requests arrive.

**Batch prediction is a workaround for when online prediction isn’t cheap enough or isn’t fast enough.**

# Unifying Batch Pipeline and Streaming Pipeline
Batch prediction is largely a product of legacy systems. In the last decade, big data processing has been dominated by batch systems like MapReduce and Spark, which allow us to periodically process a large amount of data very efficiently.

Imagine you want to build a model to predict arrival time for an application like
Google Maps. The prediction is continually updated as a user’s trip progresses. A
feature you might want to use is the average speed of all the cars in your path in the
last five minutes. For training, you might use data from the last month. To extract
this feature from your training data, you might want to put all your data into a
dataframe to compute this feature for multiple training samples at the same time.
During inference, this feature will be continually computed on a sliding window. This
means that in training this feature is computed in batch, whereas during inference
this feature is computed in a streaming process.

![unified pipeline](./screenshots/batch+streaming-pipeline.png)

# Model Compression
If the model you want to deploy takes too long to generate predictions, there are three main approaches to reduce its inference latency:
- make it do inference faster
- make the model smaller
- make the hardware it’s deployed on run faster.

The process of making a model smaller is called model compression, and the process to make it do inference faster is called inference optimization.

### Low-Rank Factorization
The key idea behind low-rank factorization is to replace high-dimensional tensors with lower-dimensional tensors.20 One type of low-rank factorization is compact convolutional filters, where the over-parameterized (having too many parameters) convolution filters are replaced with compact blocks to both reduce the number of parameters and increase speed.

For example, by using a number of strategies including replacing 3 × 3 convolution with 1 × 1 convolution, SqueezeNets achieves AlexNet-level accuracy on ImageNet with 50 times fewer parameters.


### Knowledge Distillation
Knowledge distillation is a method in which a small model (student) is trained to mimic a larger model or ensemble of models (teacher). The smaller model is what you’ll deploy.

One example of a distilled network used in production is DistilBERT, which reduces the size of a BERT model by 40% while retaining 97% of its language understanding capabilities and being 60% faster.

### Pruning
Pruning was a method originally used for decision trees where you remove sections of a tree that are uncritical and redundant for classification. As neural networks gained wider adoption, people started to realize that neural networks are over-parameterized and began to find ways to reduce the workload caused by the extra parameters.

Pruning, in the context of neural networks, has two meanings.
- One is to remove entire nodes of a neural network, which means changing its architecture and reducing its number of parameters.
- The more common meaning is to find parameters least useful to predictions and set them to 0.

### Quantization
Quantization reduces a model’s size by using fewer bits to represent its parameters. By default, most software packages use 32 bits to represent a float number. If a model has 100M parameters and each requires 32 bits to store, it’ll take up 400 MB. If we use 16 bits to represent a number, we’ll reduce the memory footprint by half. Using 16 bits to represent a float is called half precision.

Quantization not only reduces memory footprint but also improves the computation speed.
- First, it allows us to increase our batch size.
- Second, less precision speeds up computation, which further reduces training time and inference latency.

There are downsides to quantization. Reducing the number of bits to represent your numbers means that you can represent a smaller range of values. For values outside that range, you’ll have to round them up and/or scale them to be in range. Rounding numbers leads to rounding errors, and small rounding errors can lead to big performance changes.

Recently, low-precision training has become increasingly popular, with support from most modern training hardware. NVIDIA introduced Tensor Cores, processing units that support mixed-precision training.

# ML on the Cloud and on the Edge
On the cloud means a large chunk of computation is done on the cloud, either public clouds or private clouds. On the edge means a large chunk of computation is done on consumer devices—such as browsers, phones, laptops, smartwatches, cars, security cameras, robots, embedded devices, FPGAs (field programmable gate arrays), and ASICs (application-specific integrated circuits)—which are also known as edge devices.

Downsides of the cloud deployment:
- Cost
- Requires a stable internet connection to be used in the application
- Sensitive data
- ...

To move computation to the edge, the edge devices have to be powerful enough to handle the computation, have enough memory to store ML models and load them into memory, as well as have enough battery or be connected to an energy source to power the application for a reasonable amount of time. Running a full-sized BERT on your phone, if your phone is capable of running BERT, is a very quick way to kill its battery.


# Model optimization

### Vectorization
Given a loop or a nested loop, instead of executing it one item at a time, executemultiple elements contiguous in memory at the same time to reduce latency caused by data I/O.

### Parallelization
Given an input array (or n-dimensional array), divide it into different, independent work chunks, and do the operation on each chunk individually.

### Loop tiling
Change the data accessing order in a loop to leverage hardware’s memory layout and cache. This kind of optimization is hardware dependent. A good access pattern on CPUs is not a good access pattern on GPUs.

### Operator fusion
Fuse multiple operators into one to avoid redundant memory access. For example, two operations on the same array require two loops over that array.